In [1]:
# Install required packages
!pip -q install ultralytics boxmot==15.0.2 tqdm opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.9/281.9 kB 18.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, csv, math, time
import cv2, torch, numpy as np
from dataclasses import dataclass
from typing import Optional, Sequence, Tuple, List
from pathlib import Path
from tqdm.notebook import tqdm
from ultralytics import YOLO
from boxmot import StrongSort

In [ ]:
@dataclass
class YoloCfg:
    conf: float = 0.35
    iou: float = 0.5
    imgsz: int = 1280
    max_det: int = 300
    classes: Optional[Sequence[int]] = None   # e.g. [0] for COCO 'person'
    agnostic_nms: bool = False
    verbose: bool = False

@dataclass
class StrongSortCfg:
    # Only applied if attributes exist on your StrongSort build (safe setattr).
    max_age: Optional[int] = 40           # keep ID alive across occlusion
    n_init: Optional[int] = 3             # frames before confirming a track
    max_iou_dist: Optional[float] = 0.7   # motion/overlap gating
    max_dist: Optional[float] = 0.2       # appearance/cosine distance gating
    nn_budget: Optional[int] = 100        # feature queue size
    half: bool = True                     # use FP16 on GPU if available

In [ ]:
def apply_tracker_tweaks(tracker, cfg: StrongSortCfg):
    # Apply only if attribute exists in this BoxMOT version
    for k in ["max_age", "n_init", "max_iou_dist", "max_dist", "nn_budget"]:
        v = getattr(cfg, k)
        if v is not None and hasattr(tracker, k):
            setattr(tracker, k, v)

def load_models(yolo_weights: str, reid_weights: str, ssort_cfg: StrongSortCfg):
    device = 0 if torch.cuda.is_available() else 'cpu'
    tracker = StrongSort(
        reid_weights=Path(reid_weights),
        device=device,
        half=(ssort_cfg.half and torch.cuda.is_available())
    )
    apply_tracker_tweaks(tracker, ssort_cfg)
    yolo = YOLO(yolo_weights)
    return yolo, tracker

def yolo_detect(yolo: YOLO, frame: np.ndarray, cfg: YoloCfg):
    r = yolo.predict(
        frame,
        conf=cfg.conf,
        iou=cfg.iou,
        imgsz=cfg.imgsz,
        max_det=cfg.max_det,
        classes=cfg.classes,
        agnostic_nms=cfg.agnostic_nms,
        verbose=cfg.verbose
    )[0]
    if r.boxes is None or r.boxes.xyxy.numel() == 0:
        return None
    boxes = r.boxes.xyxy.detach().cpu().numpy()  # [N,4]
    confs = r.boxes.conf.detach().cpu().numpy()  # [N]
    clss  = r.boxes.cls.detach().cpu().numpy()   # [N]
    dets  = np.concatenate([boxes, confs[:, None], clss[:, None]], axis=1)
    return dets  # [x1,y1,x2,y2,conf,cls]

def draw_tracks(frame: np.ndarray, tracks: np.ndarray, id_color_cache: dict):
    # tracks: Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
    for tb in tracks:
        x1, y1, x2, y2, tid, conf, cls, _ = tb
        x1,y1,x2,y2,tid = int(x1),int(y1),int(x2),int(y2),int(tid)
        color = id_color_cache.setdefault(tid, (37*tid%256, 17*tid%256, 93*tid%256))
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

def export_tracks_csv(csv_path: str, rows: List[Tuple[int,int,int,int,int,int,float,int]]):
    # rows: (frame, id, x1,y1,x2,y2, conf, cls)
    if not csv_path: return
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["frame","id","x1","y1","x2","y2","conf","cls"])
        w.writerows(rows)

In [ ]:
def track_video(
    src_path: str,
    dst_path: str,
    yolo: YOLO,
    tracker: StrongSort,
    yolo_cfg: YoloCfg,
    export_csv: Optional[str] = None,
    show_pbar: bool = True,
):
    cap = cv2.VideoCapture(src_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {src_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w, h = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter(dst_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None
    pbar = tqdm(total=total, unit="frame", disable=not show_pbar)

    csv_rows = []
    id_color_cache = {}
    frame_idx = 0

    t0 = time.time()
    while True:
        ok, frame = cap.read()
        if not ok: break

        dets = yolo_detect(yolo, frame, yolo_cfg)
        if dets is None:
            out.write(frame)
            frame_idx += 1
            pbar.update(1)
            continue

        tracks = tracker.update(dets, frame)  # -> Nx [x1,y1,x2,y2,track_id,conf,cls,ind]
        if tracks is not None and len(tracks):
            # collect CSV rows
            for x1,y1,x2,y2,tid,conf,cls,_ in tracks:
                csv_rows.append((frame_idx, int(tid), int(x1), int(y1), int(x2), int(y2), float(conf), int(cls)))
            # draw
            frame = draw_tracks(frame, tracks, id_color_cache)

        out.write(frame)
        frame_idx += 1
        pbar.update(1)

    cap.release(); out.release()
    pbar.close()
    t1 = time.time()
    print(f"Saved video to: {dst_path} | frames={frame_idx} | time={t1-t0:.1f}s | ~{frame_idx/max(t1-t0,1):.1f} FPS")
    if export_csv:
        export_tracks_csv(export_csv, csv_rows)
        print(f"Saved track CSV to: {export_csv}")

In [ ]:
ssort_cfg = StrongSortCfg(
    max_age=50,          # keep IDs alive over occlusions/net crossings
    n_init=3,
    max_iou_dist=0.7,
    max_dist=0.25,
    nn_budget=120,
    half=True
)
yolo_cfg = YoloCfg(
    conf=0.30,           # a bit lower to catch partial players
    iou=0.55,
    imgsz=1280,
    max_det=200,
    classes=None,        # e.g., [0] if your model uses COCO 'person'
    agnostic_nms=False,
    verbose=False
)

In [ ]:
# Define model paths and input/output paths

REID_MODEL_PATH = "/content/badminton_reid_model/osnet_x1_0_badminton.pt"
YOLO_MODEL_PATH = "/content/drive/MyDrive/FIT3163,3164/YOLO/my_yolov8_1.pt"
INPUT_VIDEO_PATH = "/content/drive/MyDrive/FIT3163,3164/REID/test/reid_test_vid_1.mp4"
OUTPUT_VIDEO_PATH = "/content/out_track.mp4"
CSV_PATH = "/content/out_track.csv"

In [ ]:
yolo_model, ssort_tracker = load_models(YOLO_MODEL_PATH, REID_MODEL_PATH, ssort_cfg)

track_video(INPUT_VIDEO_PATH, OUTPUT_VIDEO_PATH, yolo_model, ssort_tracker, yolo_cfg, export_csv=CSV_PATH, show_pbar=True)

2025-08-19 07:48:36.734 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v15.0.1 🚀 Python-3.11.13 torch-2.6.0+cu124CPU
2025-08-19 07:48:36.736 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at /content/badminton_reid_model/osnet_x1_0_badminton.pt; skipping download.
2025-08-19 07:48:37.003 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from /content/badminton_reid_model/osnet_x1_0_badminton.pt


  0%|          | 0/466 [00:00<?, ?frame/s]

Saved video to: /content/out_track.mp4 | frames=466 | time=471.9s | ~1.0 FPS
Saved track CSV to: /content/out_track.csv
